In [52]:
import pandas as pd
import numpy as np
answers = pd.read_csv('./data/Answer.csv')
answers

,AnswerText,SurveyID,UserID,QuestionID
0,37,2014,1,1
1,44,2014,2,1
2,32,2014,3,1
3,31,2014,4,1
4,31,2014,5,1
...,...,...,...,...
236893,Other,2016,2689,117
236894,Support,2016,2690,117
236895,Back-end Developer,2016,2691,117
236896,DevOps/SysAdmin,2016,2692,117


In [53]:
questions = pd.read_csv('./data/Question.csv')
questions.rename(columns={'questionid': 'QuestionID'}, inplace=True)
questions

,questiontext,QuestionID
0,What is your age?,1
1,What is your gender?,2
2,What country do you live in?,3
3,"If you live in the United States, which state ...",4
4,Are you self-employed?,5
...,...,...
100,Do you think that team members/co-workers woul...,114
101,"If yes, what condition(s) have you been diagno...",115
102,"If maybe, what condition(s) do you believe you...",116
103,Which of the following best describes your wor...,117


In [54]:
surveys = pd.read_csv('./data/Survey.csv')
surveys

,SurveyID,Description
0,2014,mental health survey for 2014
1,2016,mental health survey for 2016
2,2017,mental health survey for 2017
3,2018,mental health survey for 2018
4,2019,mental health survey for 2019


## Negative age values
Some age values are negative, there fore those entries will be changed to positive.

In [55]:
answers.loc[answers['QuestionID'] == 1, 'AnswerText'] = answers.AnswerText.str.replace('-','')
answers

,AnswerText,SurveyID,UserID,QuestionID
0,37,2014,1,1
1,44,2014,2,1
2,32,2014,3,1
3,31,2014,4,1
4,31,2014,5,1
...,...,...,...,...
236893,Other,2016,2689,117
236894,Support,2016,2690,117
236895,Back-end Developer,2016,2691,117
236896,DevOps/SysAdmin,2016,2692,117


In [59]:
survey_answers = pd.merge(answers, questions, how="inner", on="QuestionID")
survey_answers = survey_answers.drop("QuestionID", 1)
has_duplicates = survey_answers['UserID'].duplicated().any()
print(has_duplicates)
test = np.where(survey_answers["SurveyID"] == 2016)
test

True


(array([  1260,   1261,   1262, ..., 236895, 236896, 236897]),)